In [1]:
import sys
import os
from warnings import simplefilter

# Add the Functions folder to your path
sys.path.append(os.path.abspath("/Users/alexandra/Desktop/EG-CLR/Functions"))

# Now import your functions
from QC_functions import *
from CLR_functions import *
from Verfication_functions import *

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)


# Run EG_CLR

In [2]:
# Load multimodal data
mdata = mu.read("/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/processed_data/multiome_data_5.25.25.h5mu")

adata_gem = mdata['rna']
adata_atac = mdata['atac']

/opt/anaconda3/envs/k562_analysis/lib/python3.11/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/opt/anaconda3/envs/k562_analysis/lib/python3.11/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [3]:
# obtaining orginal read counts
adata_org = sc.read_10x_mtx('/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix', gex_only = False)

gex_rows = list(map(lambda x: x == 'Gene Expression', adata_org.var['feature_types']))
atac_rows = list(map(lambda x: x == 'Peaks', adata_org.var['feature_types']))

adata_gem_org = adata_org[:, gex_rows].copy()
adata_atac_org = adata_org[:, atac_rows].copy()

adata_gem_org.var_names_make_unique()
adata_atac_org.var_names_make_unique()

# replace read counts with orgaianl read counts
adata_gem_new = replace_by_origin(adata_gem,adata_gem_org)
adata_atac_new = replace_by_origin(adata_atac,adata_atac_org)

In [4]:
# normalizing scRNA-seq again  
adata_gem_new.layers['counts'] = adata_gem_new.X.copy()
sc.pp.normalize_total(adata_gem_new, target_sum=1e4)
sc.pp.log1p(adata_gem_new)

# assign chr to gene
adata_gem_new = assign_chr('/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix/features.tsv.gz', adata_gem_new)

# isolate Chrx gene
adata_gem_chrx = subset_adata_by_chromosome(adata_gem_new,-1)
adata_gem_chrx

AnnData object with n_obs × n_vars = 10292 × 518
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'mt_outlier', 'S_score', 'G2M_score', 'phase'
    var: 'gene_ids', 'feature_types', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'Chromosome', 'Start', 'End'
    uns: 'log1p'
    layers: 'counts'

In [5]:
# normalizing scATAC-seq again  
adata_atac_new.layers["counts"] = adata_atac_new.X.copy()
ac.pp.tfidf(adata_atac_new, scale_factor=1e4)


# assign promoter, enhancer and gene body regions
adata_atac_new = assign_loc('/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix/features.tsv',
                            "/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/atac_peak_annotation.tsv",
                            adata_atac_new)

# assign Chr to gene
adata_atac_new = assign_chr('/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix/features.tsv.gz',
                            adata_atac_new)

# isolate Chrx peaks
adata_atac_chrx = subset_adata_by_chromosome(adata_atac_new,-1)
adata_atac_chrx

AnnData object with n_obs × n_vars = 10292 × 3424
    obs: 'n_fragment_by_counts', 'total_fragment_counts', 'log_total_fragment_counts', 'nucleosome_signal', 'tss_score', 'outlier'
    var: 'gene_ids', 'feature_types', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'gene', 'distance', 'peak_type', 'Chromosome', 'Start', 'End'
    layers: 'counts'

In [ ]:
# evalue the total overlap of promoter and expressed mRNA 
'''
total_shared_genes = 0
total_genes = 0 

for i in range(-1,23):
    adata_gem_chrx = subset_adata_by_chromosome(adata_gem_new,i)
    adata_atac_chrx = subset_adata_by_chromosome(adata_atac_new,i)
    adata_rna_flitered, adata_atac_promoter_filtered = define_rna_promoter_OCregion(adata_gem_chrx,adata_atac_chrx)
    
    print("in Chromosome", i, " :", adata_gem_chrx.shape[1], "genes and ", adata_atac_chrx.shape[1], "peaks")
    print(adata_rna_flitered.shape[1], " genes with accessible promoter and expressed mRNA" )
    
    total_genes += adata_gem_chrx.shape[1]
    total_shared_genes += adata_rna_flitered.shape[1]

print("Shared gene percentage to total gene: ", total_shared_genes/total_genes)
'''

In [ ]:
# Speed Test for different number of cells
'''
adata_gem_chrx_100, adata_atac_chrx_100 = random_subset_adatas(adata_gem_chrx, adata_atac_chrx, 100, random_state=42)
'''

In [7]:
# Option 1:  limited gene to be accessible in gene bodd and expressed mRNA
adata_rna_flitered, adata_atac_promoter_filtered = define_rna_promoter_OCregion(adata_gem_chrx,adata_atac_chrx)

combine_promoter_gene_count = combine_promoter_gene_counts(adata_atac_promoter_filtered)
combine_promoter_gene_count

AnnData object with n_obs × n_vars = 10292 × 377

In [6]:
# Option 2: limited gene to be accessible in gene body, promoter and expressed mRNA
adata_rna_flitered, adata_atac_gene_filtered, adata_CRE, adata_atac_promoter_filtered = define_rna_promoter_gene_OCregion(adata_gem_chrx, adata_atac_chrx)

combine_promoter_gene_count = combine_promoter_gene_counts(adata_atac_promoter_filtered, adata_atac_gene_filtered)
combine_promoter_gene_count

AnnData object with n_obs × n_vars = 10292 × 51

In [ ]:
# Step 1: compute MI matrix ChrX (mRNA vs ATAC peaks)
MI_Matrix_chrx = MI_Matrix_MIinfoRegression(adata_rna_flitered, adata_atac_chrx, n_neighbors=3, n_jobs=2, random_state=123)

write_matrix("/Users/alexandra/Desktop/EG-CLR/Test/results/Chr3_51Genes_S1_Binarized_10K_K562_5.28.25", MI_Matrix_chrx)

# adjust for distance 
MI_Matrix_adjusted = distance_adjustment(MI_Matrix_chrx, adata_rna_flitered.var['Start'])

# computer CLR matrix ChrX
CLR_matrix_chrx = CLR_Matrix(MI_Matrix_adjusted)


In [8]:
# Step 2: compute MI matrix ChrX (ATAC gene+promoter vs ATAC peaks)
MI_Matrix_chrx_2 = MI_Matrix_MIinfoClassif(combine_promoter_gene_count, adata_atac_chrx, n_neighbors=3, n_jobs=2, random_state=123)

write_matrix("/Users/alexandra/Desktop/EG-CLR/Test/results/Chr3_51Genes_S2_Binarized_10K_K562_5.28.25", MI_Matrix_chrx_2)

# adjust for distance 
MI_Matrix_adjusted_2 = distance_adjustment(MI_Matrix_chrx_2, adata_rna_flitered.var['Start'])

# computer CLR matrix ChrX
CLR_matrix_chrx_2 = CLR_Matrix(MI_Matrix_adjusted_2)

Calculating mutual information for ATAC peak 10/7429
Calculating mutual information for ATAC peak 20/7429
Calculating mutual information for ATAC peak 30/7429
Calculating mutual information for ATAC peak 40/7429
Calculating mutual information for ATAC peak 50/7429
Calculating mutual information for ATAC peak 60/7429
Calculating mutual information for ATAC peak 70/7429
Calculating mutual information for ATAC peak 80/7429
Calculating mutual information for ATAC peak 90/7429
Calculating mutual information for ATAC peak 100/7429
Calculating mutual information for ATAC peak 110/7429
Calculating mutual information for ATAC peak 120/7429
Calculating mutual information for ATAC peak 130/7429
Calculating mutual information for ATAC peak 140/7429
Calculating mutual information for ATAC peak 150/7429
Calculating mutual information for ATAC peak 160/7429
Calculating mutual information for ATAC peak 170/7429
Calculating mutual information for ATAC peak 180/7429
Calculating mutual information for AT

/Users/alexandra/Desktop/EG-CLR/Functions/CLR_functions.py:432: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  MI_Matrix_adjusted[gene_name][peak_name] = MI_adjusted


In [ ]:
# Step 3: compute intesection of CLR matrixes

# apply z-score filtering
CLR_matrix_chrx_z = z_score_filter(CLR_matrix_chrx, zthre=2)
CLR_matrix_chrx_2_z = z_score_filter(CLR_matrix_chrx_2, zthre=2)


# intersection of CLR matrixes
CLR_matrix_intersection = intersection_CLR(CLR_matrix_chrx_z, CLR_matrix_chrx_2_z)